In [2]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
%matplotlib inline

from importlib.util import find_spec
if find_spec("qml_hep_lhc") is None:
    import sys
    sys.path.append('..')

In [55]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Flatten, Layer
import sympy
from tensorflow import string
import tensorflow as tf
import tensorflow_quantum as tfq
import cirq
import numpy as np
from qml_hep_lhc.encodings import AngleMap
from tensorflow import Variable, random_uniform_initializer, constant, shape, repeat, tile, concat, gather

In [56]:
from qml_hep_lhc.data import ElectronPhoton, MNIST
import argparse

In [94]:
args = argparse.Namespace()

# args.graph_conv = True
# args.quantum = True
# args.pca = 64
args.resize = [4,4]
args.binary_data = [3,6]
# args.labels_to_categorical = True
# args.percent_samples = 0.01
args.min_max = True
# args.center_crop = 0.2
# args.normalize = True
# args.min_max = True
# args.threshold = 0
# args.loss = "Hinge"
# args.hinge_labels = True

In [95]:
data = MNIST(args)
data.prepare_data()
data.setup()

Binarizing data...
Resizing data...
Min-max scaling...
Binarizing data...
Resizing data...
Min-max scaling...


In [96]:
data

Dataset :MNIST
╒════════╤══════════════════╤═════════════════╤═══════════╕
│ Data   │ Train size       │ Test size       │ Dims      │
╞════════╪══════════════════╪═════════════════╪═══════════╡
│ X      │ (12049, 4, 4, 1) │ (1968, 4, 4, 1) │ (4, 4, 1) │
├────────┼──────────────────┼─────────────────┼───────────┤
│ y      │ (12049,)         │ (1968,)         │ (1,)      │
╘════════╧══════════════════╧═════════════════╧═══════════╛

Train images stats
Min: -3.14
Max: 3.14
Mean: -2.43
Std: 1.81
Train labels stats
Min: 0.00
Max: 1.00

In [97]:
class QLinear(Layer):
    def __init__(self,input_dim):
        super(QLinear, self).__init__()
        
        # Prepare qubits
        self.n_qubits = np.prod(input_dim)
        self.qubits = cirq.GridQubit.rect(1, self.n_qubits)
        
        self.readout = cirq.GridQubit(-1, -1)
        self.observables = [cirq.Z(self.readout)]
        
        var_symbols = sympy.symbols(f'qnn0:{2*self.n_qubits}')
        self.var_symbols = np.asarray(var_symbols).reshape((self.n_qubits, 2))

        in_symbols = sympy.symbols(f'x0:{self.n_qubits}')
        self.in_symbols = np.asarray(in_symbols).reshape((self.n_qubits))
    
    def build(self,input_shape):
        circuit = cirq.Circuit()

        # Prepare the readout qubit
        circuit.append(cirq.X(self.readout))
        circuit.append(cirq.H(self.readout))

        fm = AngleMap()
        circuit += fm.build(self.qubits, self.in_symbols)
        
        for i, qubit in enumerate(self.qubits):
            circuit.append(cirq.XX(qubit, self.readout)**self.var_symbols[i, 0])
        for i, qubit in enumerate(self.qubits):
            circuit.append(cirq.ZZ(qubit, self.readout)**self.var_symbols[i, 1])

        # Finally, prepare the readout qubit.
        circuit.append(cirq.H(self.readout))

        self.var_symbols = list(self.var_symbols.flat)
        self.in_symbols = list(self.in_symbols.flat)

        var_init = random_uniform_initializer(minval=-np.pi / 2, maxval=np.pi / 2)
        self.theta = Variable(initial_value=var_init(
            shape=(1, len(self.var_symbols)), dtype="float32"),
                              trainable=True,
                              name="thetas")

        # Define explicit symbol order
        symbols = [str(symb) for symb in self.var_symbols + self.in_symbols]
        self.indices = constant([symbols.index(a) for a in sorted(symbols)])

        self.empty_circuit = tfq.convert_to_tensor([cirq.Circuit()])
        self.computation_layer = tfq.layers.ControlledPQC(circuit, self.observables)
        
    
    def call(self,input_tensor):
        batch_dim = shape(input_tensor)[0]
        
        tiled_up_circuits = repeat(self.empty_circuit,
                                   repeats=batch_dim,
                                   name="tiled_up_circuits")
        tiled_up_thetas = tile(self.theta,
                               multiples=[batch_dim, 1],
                               name="tiled_up_thetas")
        joined_vars = concat([tiled_up_thetas, input_tensor], axis=-1)
        joined_vars = gather(joined_vars,
                             self.indices,
                             axis=-1,
                             name='joined_vars')
        out = self.computation_layer([tiled_up_circuits, joined_vars])
        return out

In [98]:
class QNNNEW(Model):
    """
    Quantum Neural Network.
    This implementation is based on https://www.tensorflow.org/quantum/tutorials/mnist
    """

    def __init__(self, data_config, args=None):
        super().__init__()
        self.args = vars(args) if args is not None else {}

        # Data config
        self.input_dim = data_config["input_dims"]
        self.qlinear = QLinear(self.input_dim)


    def call(self, input_tensor):
        """
        The function takes in an input tensor and returns the expectation of the input tensor
        
        Args:
          input_tensor: The input tensor to the layer.
        
        Returns:
          The expectation of the input tensor.
        """
        x = Flatten()(input_tensor)
        out = self.qlinear(x)
        return out
        

    def build_graph(self):
        # x = Input(shape=(), dtype=string)
        x = Input(shape=self.input_dim)
        return Model(inputs=[x], outputs=self.call(x), name="QNN")

    @staticmethod
    def add_to_argparse(parser):
        return parser

In [99]:
# model = QCNNChen(data.config())
model = QNNNEW({'input_dims':(4,4,1)})

In [100]:
model.build_graph().summary()

Model: "QNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 4, 4, 1)]         0         
                                                                 
 flatten_6 (Flatten)         (None, 16)                0         
                                                                 
 q_linear_4 (QLinear)        (None, 1)                 32        
                                                                 
Total params: 32
Trainable params: 32
Non-trainable params: 0
_________________________________________________________________


In [101]:
loss_fn = tf.keras.losses.MeanSquaredError
optimizer = tf.keras.optimizers.Adam

In [102]:
@tf.function
def custom_accuracy(y_true, y_pred):
    y_true = tf.squeeze(y_true)
    y_pred = tf.map_fn(lambda x: 1.0 if x >= 0 else -1.0, y_pred)
    return tf.keras.backend.mean(tf.keras.backend.equal(y_true, y_pred))


In [103]:
model.compile(loss=loss_fn(), metrics=[custom_accuracy], optimizer=optimizer(learning_rate=0.001))

In [104]:
model.fit(data.x_train,
          data.y_train, 
          batch_size=32,
          epochs=3,
          validation_split=0.2,
          shuffle=True,
          workers=4)

Epoch 1/3
302/302 [==============================] - 506s 2s/step - loss: 0.3605 - custom_accuracy: 0.4281 - val_loss: 0.2815 - val_custom_accuracy: 0.4705
Epoch 2/3
 73/302 [======>.......................] - ETA: 11:02 - loss: 0.2407 - custom_accuracy: 0.4765

KeyboardInterrupt: 